# Climate Analysis

Using Python and SQLAlchemy again, basic climate analysis and data exploration will be completed within this notebook. Tools used will be ORM queries via SQLAlchemy, Matplotlib, and Pandas. 

Analysis will be performed on precipitation, temperature, and stations.

In [34]:
# Import dependencies
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

### Setting up and reflecting the existing sqlite database

In [35]:
# Database setup
engine = create_engine("sqlite:///hawaii.sqlite")

In [36]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [37]:
# Save reference to the tables
Measurement = Base.classes.measurements
Stations = Base.classes.stations

In [38]:
# Create our session (link) from Python to the DB
session = Session(bind = engine)

### Climate Analysis: precipitation

In [40]:
# Select prcp & date for the last 12 months
results = session.query(Measurement.date, Measurement.precip).filter(Measurement.date >= '2017-01-01')

In [41]:
# create empty dict to populate
data = {'date': [], 'prcp': []}

# populate dict with rows from results
for row in results:
    data['date'].append(row.date)
    data['prcp'].append(row.precip)

# create pandas df from dict
precip17 = pd.DataFrame(data['prcp'], columns = ['precipitation'], index = data['date'])
print(prcp17.head())

Empty DataFrame
Columns: [precipitation]
Index: []


In [42]:
precip17

,precipitation


### Climate Analysis: temperature

In [43]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
print(calc_temps('2012-02-28', '2012-03-05'))

[(None, None, None)]


### Climate Analysis: stations

In [ ]:
# Total number of stations
# Most active stations
### List stations and obs counts in descending order
### Which station has highest number of observations
# Retrieve last 12 months of temp obs data

In [44]:
# Flask Setup
app = Flask(__name__)